<a href="https://colab.research.google.com/github/LorenaGiVa/Caso---Marketing/blob/main/b_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import plotly.graph_objs as go
import plotly.express as px


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
##### conectarse a BD #######
conn= sql.connect('/content/drive/MyDrive/Caso marketing analítica/data/db_movies')
cur=conn.cursor()

In [4]:
### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',)]

In [5]:
########### traer tabla de BD a python ####
movies= pd.read_sql("select *  from movies", conn)
ratings = pd.read_sql('select * from ratings', conn)

In [6]:
### ver el tipo de datos y faltantes
print(movies.info())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


No hay nulos

In [7]:
### datos duplicados
print(movies.duplicated().sum())
print(ratings.duplicated().sum())

0
0


In [8]:
# distribuciin de las calificaciones
cr=pd.read_sql(""" select rating,
                    count(*) as conteo
                    from ratings
                    group by rating
                    order by conteo desc""", conn)

In [9]:
data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)

In [10]:
### calcular cada usuario cuantas peliculas calificó
rating_users = pd.read_sql(''' select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                order by cnt_rat asc
                                ''',conn )

In [11]:
rating_users = pd.read_sql(''' select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                order by cnt_rat asc
                                ''',conn )

In [12]:
rating_users

,userId,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


In [13]:
fig  = px.histogram(rating_users, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [14]:
rating_users.describe()

,userId,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


In [15]:
rating_users2 = pd.read_sql('''select userId,
                                count(*) as cnt_rat
                                from ratings
                                group by userId
                                having cnt_rat <=1000
                                order by cnt_rat asc
                                ''',conn )

In [16]:
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [17]:
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por usuario')
fig.show()

In [18]:
### calificacion de cada pelicula
rating_movie = pd.read_sql('''select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                order by cnt_rat desc
                                ''',conn )


In [19]:
fig  = px.histogram(rating_movie, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [20]:
rating_movie.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


In [21]:
####peliculas que tengan más de 5 calificaciones
rating_movie2=pd.read_sql(''' select movieId,
                                count(*) as cnt_rat
                                from ratings
                                group by movieId
                                having cnt_rat >= 5
                                order by cnt_rat desc
                                ''',conn )

In [22]:
fig  = px.histogram(rating_movie2, x= 'cnt_rat', title= 'Histograma frecuencia de número de calificaciones por pelicula')
fig.show()

In [23]:
rating_movie2.describe()

,movieId,cnt_rat
count,3650.000000,3650.000000
mean,23839.433151,24.732603
std,38394.748531,31.703576
min,1.000000,5.000000
25%,1784.250000,7.000000
50%,3866.000000,13.000000
75%,36664.750000,28.000000
max,187595.000000,329.000000


In [24]:
###### para ejecutar sql y conectarse a bd ###

## crear copia de db_books datos originales, nombrarla books2 y procesar books2

conn=sql.connect('/content/drive/MyDrive/Caso marketing analítica/data/db_movies2.db') ### crear cuando no existe el nombre de cd  y para conectarse cuando sí existe.
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [25]:
#### para consultar datos ######## con cur
cur.execute("select * from full_ratings")
cur.fetchall()

[(1,
  1,
  4.0,
  'Toy Story (1995)',
  'Adventure|Animation|Children|Comedy|Fantasy',
  'Toy Story (1995)',
  1995,
  '2000-07-30'),
 (1,
  3,
  4.0,
  'Grumpier Old Men (1995)',
  'Comedy|Romance',
  'Grumpier Old Men (1995)',
  1995,
  '2000-07-30'),
 (1,
  6,
  4.0,
  'Heat (1995)',
  'Action|Crime|Thriller',
  'Heat (1995)',
  1995,
  '2000-07-30'),
 (1,
  47,
  5.0,
  'Seven (a.k.a. Se7en) (1995)',
  'Mystery|Thriller',
  'Seven (a.k.a. Se7en) (1995)',
  1995,
  '2000-07-30'),
 (1,
  50,
  5.0,
  'Usual Suspects, The (1995)',
  'Crime|Mystery|Thriller',
  'Usual Suspects, The (1995)',
  1995,
  '2000-07-30'),
 (1,
  70,
  3.0,
  'From Dusk Till Dawn (1996)',
  'Action|Comedy|Horror|Thriller',
  'From Dusk Till Dawn (1996)',
  1996,
  '2000-07-30'),
 (1,
  101,
  5.0,
  'Bottle Rocket (1996)',
  'Adventure|Comedy|Crime|Romance',
  'Bottle Rocket (1996)',
  1996,
  '2000-07-30'),
 (1,
  110,
  4.0,
  'Braveheart (1995)',
  'Action|Drama|War',
  'Braveheart (1995)',
  1995,
  '2000

In [26]:
##### consultar trayendo para pandas ###
df_final=pd.read_sql("select * from full_ratings", conn)
df_final

,user_id,movie_id,rating,movie_title,movie_genres,movie_clean_title,movie_year,fecha_nueva
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995),1995.0,2000-07-30
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995),1995.0,2000-07-30
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,Heat (1995),1995.0,2000-07-30
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven (a.k.a. Se7en) (1995),1995.0,2000-07-30
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The (1995)",1995.0,2000-07-30
...,...,...,...,...,...,...,...,...
70524,609,892,3.0,Twelfth Night (1996),Comedy|Drama|Romance,Twelfth Night (1996),1996.0,1996-11-05
70525,609,1056,3.0,Jude (1996),Drama,Jude (1996),1996.0,1996-11-05
70526,609,1059,3.0,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,William Shakespeare's Romeo + Juliet (1996),1996.0,1996-11-05
70527,609,1150,4.0,"Return of Martin Guerre, The (Retour de Martin...",Drama,"Return of Martin Guerre, The (Retour de Martin...",1982.0,1996-11-05


In [27]:
# Convertir la columna 'fecha_nueva' a tipo str y eliminar el '.0' al final
df_final['movie_year'] = df_final['movie_year'].astype(str).str[:-2]

# Mostrar el DataFrame actualizado
df_final

,user_id,movie_id,rating,movie_title,movie_genres,movie_clean_title,movie_year,fecha_nueva
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995),1995,2000-07-30
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995),1995,2000-07-30
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,Heat (1995),1995,2000-07-30
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven (a.k.a. Se7en) (1995),1995,2000-07-30
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The (1995)",1995,2000-07-30
...,...,...,...,...,...,...,...,...
70524,609,892,3.0,Twelfth Night (1996),Comedy|Drama|Romance,Twelfth Night (1996),1996,1996-11-05
70525,609,1056,3.0,Jude (1996),Drama,Jude (1996),1996,1996-11-05
70526,609,1059,3.0,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,William Shakespeare's Romeo + Juliet (1996),1996,1996-11-05
70527,609,1150,4.0,"Return of Martin Guerre, The (Retour de Martin...",Drama,"Return of Martin Guerre, The (Retour de Martin...",1982,1996-11-05
